In [ ]:
pip install pyspark py4j


In [57]:
from pyspark.sql import SparkSession

#Crear sesion de Spark
spark = SparkSession.builder\
        .master("local[*]")\
        .appName('nubi_chall')\
        .getOrCreate()


In [58]:
import os
from os import truncate
from google.colab import drive
from pyspark.sql.types import  StructField, DoubleType, IntegerType, StringType, StructType, LongType
from pyspark.sql.window import Window
from pyspark.sql.functions import col, when, sum, count, rank, lit

#Montar Google Drive
drive.mount('/content/drive')
#spark.conf.set("spark.sql.parquet.int64AsDouble", "true")



#1. Lectura de los archivos de los primeros 7 días del mes. En caso de haber días faltantes en el período analizado, logear dichas fechas para identificar cuáles son los faltantess.

#Ruta base de las carpetas
df1 = spark.read.parquet("/content/drive/MyDrive/compressedData/year=2024/month=07/day=20240701")
df2 = spark.read.parquet("/content/drive/MyDrive/compressedData/year=2024/month=07/day=20240702")
df3 = spark.read.parquet("/content/drive/MyDrive/compressedData/year=2024/month=07/day=20240703")
df4 = spark.read.parquet("/content/drive/MyDrive/compressedData/year=2024/month=07/day=20240705")
df5 = spark.read.parquet("/content/drive/MyDrive/compressedData/year=2024/month=07/day=20240706")
df6 = spark.read.parquet("/content/drive/MyDrive/compressedData/year=2024/month=07/day=20240707")


#("/content/drive/MyDrive/compressedData/year=2024/month=07/day=20240705/sellerId=999888777/")

#df1.printSchema()
#df2.printSchema()
#df3.printSchema()
#df4.printSchema()
#df5.printSchema()
#df6.printSchema()

#Unimos los dataframes posterior a verificacion de datos de manera independiente
dfUnida = df2.unionByName(df3, allowMissingColumns=True) \
                 .unionByName(df4, allowMissingColumns=True) \
                 .unionByName(df5, allowMissingColumns=True) \
                 .unionByName(df1, allowMissingColumns=True) \
                 .unionByName(df6, allowMissingColumns=True)

#Establezco limite de 30 registros ya que no supera dicho numero
dfUnida.show(30)


#dfUnida.printSchema()  La idea era verificar el schema una vez unido para que no haya problema con el tipo de dato en columnas

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
+--------+---------+-------+-----+---------+-----------+--------------------+
|currency|       id|  price|sales| sellerId|   category|               title|
+--------+---------+-------+-----+---------+-----------+--------------------+
|     ARS|MLA123456|  150.0|    2|999888777|       NULL|                NULL|
|     ARS|MLA654321|   30.0|    1|999888777|       NULL|                NULL|
|     ARS|MLA333333|   20.0|   15|444555666|       NULL|                NULL|
|     ARS|MLA333444|   10.0|    3|444555666|       NULL|                NULL|
|     ARS|MLA123456|  160.0|    5|999888777|Electronics|                NULL|
|     ARS|MLA654321|   60.0|    1|999888777|Electronics|                NULL|
|     ARS|MLA333333|   50.0|   16|444555666|Electronics|                NULL|
|     ARS|MLA121212|   10.0|    6|444555666|Electronics|                NULL|
|     ARS|MLA

In [59]:

#dfRevisionDias = spark.read.parquet('/content/drive/MyDrive/compressedData/year=2024/month=07')
#Ruta base de las carpetas
base_path = '/content/drive/MyDrive/compressedData/year=2024/month=07/'

#Lista de días esperados del 1 al 7
diasSemana = [f'day=2024070{dia}' for dia in range(1, 8)]

#Lista de carpetas presentes en la ruta
diasEncontrados = [folder for folder in os.listdir(base_path) if folder.startswith('day=')]

#Comparar días esperados con los que existen
diasFaltantes = [dia for dia in diasSemana if dia not in diasEncontrados]

#Mostrar los resultados
if diasFaltantes:
    print(f"Faltan los siguientes días: {diasFaltantes}")
else:
    print("No faltan días.")


Faltan los siguientes días: ['day=20240704']


In [60]:

#2. Agregar una columna al dataframe leído en el punto 1 que calcule la facturación de cada venta (cada fila es una venta).
dfUnida = dfUnida.withColumn("GMV", when(col("sales").isNull() | col("price").isNull(), 0)
                                    .otherwise(col("sales") * col("price")))

dfUnida.show(30)
dfUnida.printSchema()

+--------+---------+-------+-----+---------+-----------+--------------------+-------+
|currency|       id|  price|sales| sellerId|   category|               title|    GMV|
+--------+---------+-------+-----+---------+-----------+--------------------+-------+
|     ARS|MLA123456|  150.0|    2|999888777|       NULL|                NULL|  300.0|
|     ARS|MLA654321|   30.0|    1|999888777|       NULL|                NULL|   30.0|
|     ARS|MLA333333|   20.0|   15|444555666|       NULL|                NULL|  300.0|
|     ARS|MLA333444|   10.0|    3|444555666|       NULL|                NULL|   30.0|
|     ARS|MLA123456|  160.0|    5|999888777|Electronics|                NULL|  800.0|
|     ARS|MLA654321|   60.0|    1|999888777|Electronics|                NULL|   60.0|
|     ARS|MLA333333|   50.0|   16|444555666|Electronics|                NULL|  800.0|
|     ARS|MLA121212|   10.0|    6|444555666|Electronics|                NULL|   60.0|
|     ARS|MLA999000|  150.0|    2|999888777|      Tool

In [61]:
#3.

#a- Calcular la facturación total de cada seller.
dfFacturacionTotal = dfUnida.groupBy("sellerId") \
                            .agg(sum("GMV").alias("total_revenue"))

dfFacturacionTotal.show()

#b- Unidades vendidas por item
dfUnidadesVendidas = dfUnida.groupBy("id") \
                        .agg(sum("sales").alias("total_units_sold"))

dfUnidadesVendidas.show()

+---------+-------------+
| sellerId|total_revenue|
+---------+-------------+
|999888777|     126401.0|
|444555666|       3924.0|
|111222333|       9075.0|
+---------+-------------+

+---------+----------------+
|       id|total_units_sold|
+---------+----------------+
|MLA123456|              10|
|MLA654321|               2|
|MLA333333|              36|
|MLA333444|               3|
|MLA991199|               2|
|MLA991188|               3|
|MLA121212|               6|
|MLA999000|               2|
|MLA111111|             121|
|MLA555555|               5|
|MLA676767|               8|
|MLA888999|              11|
|MLA777000|              11|
|MLA222333|              21|
|MLA222111|              16|
+---------+----------------+



In [62]:
#4. Asignar un ranking a cada vendedor según los totales calculados en el punto 3 a. Cada vendedor debe obtener una posición
#  en el ranking según el total facturado, siendo la posición 1 la de mayor facturación. Guardar el ranking en formato de un único CSV

dfFacturacionTotal_ranked = dfFacturacionTotal.withColumn(
    "rank",
    rank().over(Window.orderBy(col("total_revenue").desc()))  #Ordena las filas dentro de la ventana por la columna "total_revenue" de mayor a menor
)

#Filtramos y seleccionamos solamente las columnas "sellerId" y "rank" para ver resultados
dfRankingVentas = dfFacturacionTotal_ranked.select("sellerId", "rank")

#Se guarda ranking en archivo CSV, tambien estuvo en mente la descarga a maquina local del mismo pero pensandolo a gran escala no es muy optimizada la solucion.
dfRankingVentas.coalesce(1).write.csv("dfRankingVentas.csv", header=True)

dfRankingVentas.show()

AnalysisException: [PATH_ALREADY_EXISTS] Path file:/content/dfRankingVentas.csv already exists. Set mode as "overwrite" to overwrite the existing path.

In [63]:
#5. Guardar en formato parquet el dataframe del punto 2 de forma particionada por año, mes, seller.

#Aca creamos las columnas año y mes
dfUnida = dfUnida.withColumn("year", lit(2024)) \
                         .withColumn("month", lit(7))

#Particionamos por año, mes, seller, esta linea podria añadirse al inicio debido a que solamente se puede ejecutar una vez
dfUnida.write.partitionBy("year", "month", "sellerId").parquet("combined_data.parquet")




AnalysisException: [PATH_ALREADY_EXISTS] Path file:/content/combined_data.parquet already exists. Set mode as "overwrite" to overwrite the existing path.

In [64]:
#Code cell adicional para leer y mostrar archivo parquet particionado por año, mes, seller.
dfParquet= spark.read.parquet("/content/combined_data.parquet")

#Ordenamos por sellerId de mayor a menor
dfParquetOrdenado = dfParquet.orderBy(col("sellerId").desc())

dfParquetOrdenado.show(30)

+--------+---------+-------+-----+-----------+--------------------+-------+----+-----+---------+
|currency|       id|  price|sales|   category|               title|    GMV|year|month| sellerId|
+--------+---------+-------+-----+-----------+--------------------+-------+----+-----+---------+
|     ARS|MLA123456|  150.0|    2|       NULL|                NULL|  300.0|2024|    7|999888777|
|     ARS|MLA123456|  160.0|    5|Electronics|                NULL|  800.0|2024|    7|999888777|
|     ARS|MLA654321|   30.0|    1|       NULL|                NULL|   30.0|2024|    7|999888777|
|     ARS|MLA654321|   60.0|    1|Electronics|                NULL|   60.0|2024|    7|999888777|
|     ARS|MLA123456|  150.0|    3|       NULL|                NULL|  450.0|2024|    7|999888777|
|     ARS|MLA999000|  150.0|    2|      Tools|                NULL|  300.0|2024|    7|999888777|
|     ARS|MLA991199|   90.0|    2|      Shoes|                NULL|  180.0|2024|    7|999888777|
|     ARS|MLA991188|   60.0|  

In [65]:
!pip install azure-data-tables
!pip install azure-storage-blob
!pip install python-dotenv #configurar variable de entorno

In [75]:
from azure.data.tables import TableServiceClient
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

# Conexión a la cuenta de Azure Table
#The connection string was modified to use double curly braces to properly format the f-string and access the environment variable.
connection_string = f"DefaultEndpointsProtocol=https;AccountName={os.getenv('AZURE_STORAGE_ACCOUNT')};AccountKey={os.getenv('AZURE_STORAGE_KEY')};EndpointSuffix=core.windows.net"
table_service_client = TableServiceClient.from_connection_string(conn_str=connection_string)

# Crear cliente para la tabla
table_client = table_service_client.get_table_client(table_name="FacturacionVentas")

# Subir cada fila de la tabla
for row in dfUnida.collect():
    entity = {
        'PartitionKey': str(row['sellerId']),
        'RowKey': str(row['id']),
        'price': row['price'],
        'sales': row['sales'],
        'currency': row['currency'],
        'category': row['category'],
        'title': row['title'],
        'GMV': row['GMV']
    }
    try:
        # Intenta crear la entidad
        table_client.create_entity(entity=entity)
    except ResourceExistsError:
        # Maneja la excepción si la entidad ya existe
        print(f"Entity with PartitionKey '{entity['PartitionKey']}' and RowKey '{entity['RowKey']}' already exists.")
        # Puedes optar por actualizar la entidad existente o simplemente ignorar el error.
        # Para actualizar la entidad:
        # table_client.update_entity(entity=entity)

        # Consultar todas las entidades (esto puede ser un gran volumen de datos)
# Se agrega un filtro para que la consulta retorne todas las entidades
entities = table_client.query_entities(query_filter="")

for entity in entities:
    print(entity)


# Consultar entidades con un PartitionKey específico
entities = table_client.query_entities(query_filter="PartitionKey eq '999888777'")

for entity in entities:
    print(entity)

# Consultar entidades con un filtro más complejo
entities = table_client.query_entities(query_filter="PartitionKey eq '999888777' and price gt 10")

for entity in entities:
    print(entity)


Entity with PartitionKey '999888777' and RowKey 'MLA123456' already exists.
Entity with PartitionKey '999888777' and RowKey 'MLA654321' already exists.
Entity with PartitionKey '444555666' and RowKey 'MLA333333' already exists.
Entity with PartitionKey '444555666' and RowKey 'MLA333444' already exists.
Entity with PartitionKey '999888777' and RowKey 'MLA123456' already exists.
Entity with PartitionKey '999888777' and RowKey 'MLA654321' already exists.
Entity with PartitionKey '444555666' and RowKey 'MLA333333' already exists.
Entity with PartitionKey '444555666' and RowKey 'MLA121212' already exists.
Entity with PartitionKey '999888777' and RowKey 'MLA999000' already exists.
Entity with PartitionKey '999888777' and RowKey 'MLA991199' already exists.
Entity with PartitionKey '999888777' and RowKey 'MLA991188' already exists.
Entity with PartitionKey '444555666' and RowKey 'MLA555555' already exists.
Entity with PartitionKey '444555666' and RowKey 'MLA676767' already exists.
Entity with 